In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.applications import VGG16

# Function to filter out non-image files
def is_image_file(filename):
    """
    Check if a filename represents an image file.

    Args:
    - filename (str): The name of the file to check.

    Returns:
    - bool: True if the filename represents an image file, False otherwise.
    """
    return any(filename.endswith(extension) for extension in ['.png', '.jpg', '.jpeg'])

# Function to load and preprocess images
def load_and_preprocess_image(image_path, target_size):
    """
    Load and preprocess an image from a given path.

    Args:
    - image_path (str): The path to the image file.
    - target_size (tuple): The target size for the loaded image.

    Returns:
    - numpy.ndarray: The preprocessed image as a NumPy array.
    """
    img = load_img(image_path, target_size=target_size)
    img_array = img_to_array(img)
    return img_array / 255.0  # normalization

# Function to load images and labels from a folder
def load_images_from_folder(folder):
    """
    Load images and corresponding labels from a folder.

    Args:
    - folder (str): The path to the folder containing images and labels.

    Returns:
    - tuple: A tuple containing lists of image paths and corresponding labels.
    """
    images = []
    labels = []
    for label in os.listdir(folder):
        label_path = os.path.join(folder, label)
        if os.path.isdir(label_path):
            for filename in os.listdir(label_path):
                if is_image_file(filename):
                    img_path = os.path.join(label_path, filename)
                    images.append(img_path)
                    labels.append(label)
    return images, labels

# Define paths and parameters
user_home = os.path.expanduser("~")
data_folder = "D:\\car_dataset"  # dataset folder
data_path = os.path.join(user_home, data_folder)
image_size = (128, 128)
batch_size = 32
epochs = 10

# Data preprocessing
image_list = []
class_labels = []

for class_label in os.listdir(data_path):
    class_path = os.path.join(data_path, class_label)
    if os.path.isdir(class_path):
        for filename in os.listdir(class_path):
            if is_image_file(filename):
                image_path = os.path.join(class_path, filename)
                image_list.append(image_path)
                class_labels.append(class_label)

# Load images and labels from the "tesla" folder
tesla_folder = os.path.join(data_path, "tesla")
tesla_images, tesla_labels = load_images_from_folder(tesla_folder)

# Add Tesla images and labels to the existing dataset
image_list += tesla_images
class_labels += tesla_labels

# Convert images to numpy arrays
X = np.array([load_and_preprocess_image(img, image_size) for img in image_list])
y = np.array(class_labels)

# One-hot encode the labels
label_binarizer = LabelBinarizer()
y = label_binarizer.fit_transform(y)

# Split the dataset into training and validation sets
X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Data visualization - Class distribution
plt.figure(figsize=(10, 6))
sns.countplot(x=y_train.argmax(axis=1), palette='viridis')
plt.title('Class Distribution in Training Data')
plt.xlabel('Class')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show()

# Sample Data Visualization
plt.figure(figsize=(12, 8))
for i in range(12):  # Adjust the number of sample images to display
    plt.subplot(3, 4, i + 1)
    plt.imshow(X_train[i])
    plt.title('Class: ' + class_labels[np.argmax(y_train[i])])
    plt.axis('off')
plt.show()

# Model architecture (choose either VGG or ResNet)
from tensorflow.keras.applications import VGG16

# Load the VGG16 model without the top (fully connected) layers
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(image_size[0], image_size[1], 3))

# Freeze the convolutional layers
for layer in vgg_base.layers:
    layer.trainable = False

# Add custom top layers
model = Sequential()
model.add(vgg_base)
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(len(label_binarizer.classes_), activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=epochs, validation_data=(X_validation, y_validation))

# Evaluate the model
loss, accuracy = model.evaluate(X_validation, y_validation)

print("Validation Loss:", loss)
print("Validation Accuracy:", accuracy)



KeyboardInterrupt

